In [11]:
import xarray as xr
import PIL
import hvplot.xarray
import holoviews as hv
hv.extension('bokeh')
from matplotlib import pyplot as plt

In [12]:
# #extent = (139.2, 140.2, 34.8, 35.8)  ### Mouth of Tokyo Bay
# # extent = (-82.0, -65.0, 28.0, 43.0)  ### Project Sandy
# extent = (117.376709, 124.287109, -6.130845, -2.538232)  ### Project Sulawesi Selatan (Lon, Lat: http://bboxfinder.com/)
# time_start, time_end, dtime = ('2012-10-28 12:00:00', '2012-10-29 00:00:00', 3)  ### dtime (int): time interval in hours
# ### Reading catalog from local
# #xh.run_opendap(extent=extent, time_start=time_start, time_end=time_end, dtime=dtime)
# ### Reading catalog from OPeNDAP
# dirpath = 'http://tds.hycom.org/thredds/dodsC/GLBv0.08/expt_53.X/data/'
# xh.run_opendap(extent=extent, time_start=time_start, time_end=time_end, dtime=dtime, opendap=True, dirpath=dirpath)

In [13]:
inputs = 'dataset/hycom/*.nc'
output = 'output/'

with xr.open_mfdataset(inputs, parallel=True, data_vars='minimal', \
                       coords='minimal', compat='override') as dsts:
    print("Creating xarray.Dataset from " + str(inputs))
dsts

Creating xarray.Dataset from dataset/hycom/*.nc


<xarray.Dataset> Size: 51MB
Dimensions:     (time: 8, lat: 128, lon: 78, depth: 40)
Coordinates:
  * time        (time) datetime64[ns] 64B 2019-01-01T12:00:00 ... 2019-01-02T...
  * lat         (lat) float64 1kB -7.52 -7.48 -7.44 -7.4 ... -2.52 -2.48 -2.44
  * lon         (lon) float64 624B 118.4 118.5 118.6 118.6 ... 124.4 124.5 124.6
  * depth       (depth) float64 320B 0.0 2.0 4.0 6.0 ... 3e+03 4e+03 5e+03
Data variables:
    surf_el     (time, lat, lon) float32 319kB dask.array<chunksize=(1, 128, 78), meta=np.ndarray>
    water_temp  (time, depth, lat, lon) float32 13MB dask.array<chunksize=(1, 13, 128, 78), meta=np.ndarray>
    salinity    (time, depth, lat, lon) float32 13MB dask.array<chunksize=(1, 13, 128, 78), meta=np.ndarray>
    water_u     (time, depth, lat, lon) float32 13MB dask.array<chunksize=(1, 13, 128, 78), meta=np.ndarray>
    water_v     (time, depth, lat, lon) float32 13MB dask.array<chunksize=(1, 13, 128, 78), meta=np.ndarray>
Attributes: (12/15)
    classification_level:      UNCLASSIFIED
    distribution_statement:    Approved for public release. Distribution unli...
    downgrade_date:            not applicable
    classification_authority:  not applicable
    institution:               Fleet Numerical Meteorology and Oceanography C...
    source:                    HYCOM archive file
    ...                        ...
    Conventions:               CF-1.6 NAVO_netcdf_v1.1
    History:                   Translated to CF-1.0 Conventions by Netcdf-Jav...
    geospatial_lat_min:        -7.519999980926514
    geospatial_lat_max:        -2.440000057220459
    geospatial_lon_min:        118.4000244140625
    geospatial_lon_max:        124.56005859375

In [25]:
%%time
da = dsts.sel(depth=0, time='2019-01-01 15:00', method='nearest').load()

CPU times: total: 0 ns
Wall time: 23 ms


In [26]:
da.salinity.hvplot.quadmesh(x='lon', y='lat', geo=True, tiles='OSM', cmap='viridis', rasterize=True)

BokehModel(combine_events=True, render_bundle={'docs_json': {'b257ee14-aed1-4eeb-bee4-939c8109fc8d': {'version…

In [16]:
%%time
da = dsts.sel(depth=0, method='nearest').load()

CPU times: total: 125 ms
Wall time: 94.6 ms


In [17]:
hv.extension('matplotlib')

def create_gif(df, column):
    image_frames = [] # creating a empty list to be appended later on

    for i, timestep in enumerate(df.time):
        new_fram = PIL.Image.open(f"{output}/water_temp_timestep_{i}.png") 
        image_frames.append(new_fram)

    image_frames[0].save(f"{output}/{column}_timelapse.gif",format='GIF',
                        append_images = image_frames[1: ],
                        save_all = True, duration = 100,
                        loop = 0)

def create_images(df, column):
    # Iterate over each time step and save the plot
    for i, timestep in enumerate(df.time):
        # Select the current time slice
        da_slice = df[column].sel(time=timestep)
        
        # Generate the plot
        plot = da_slice.hvplot.quadmesh(
            x='lon', y='lat', geo=True, cmap='viridis', rasterize=True
        )
        
        # Save the plot as PNG
        fig = hv.render(plot, backend='matplotlib')
        fig.savefig(f"{output}/{column}_timestep_{i}.png")
        plt.close(fig)  # Close the figure after saving to free memory
        print(f"Saved: {column}_timestep_{i}.png")
    
    create_gif(df, column)

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

In [18]:
create_images(da, 'salinity')

Saved: salinity_timestep_0.png
Saved: salinity_timestep_1.png
Saved: salinity_timestep_2.png
Saved: salinity_timestep_3.png
Saved: salinity_timestep_4.png
Saved: salinity_timestep_5.png
Saved: salinity_timestep_6.png
Saved: salinity_timestep_7.png
